<a href="https://colab.research.google.com/github/yukiharada1228/neural_network/blob/main/rinna%E3%81%AB%E3%82%88%E3%82%8B%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリをインストール
!pip install -q transformers accelerate sentencepiece bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# rinnaチャットボットを作成
class RinnaChatBot:
    def __init__(self, max_length=128, k=40):
        model_id = "rinna/japanese-gpt-neox-3.6b-instruction-ppo"
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto"
        )
        self.log = []
        self.max_length = max_length
        self.k = k

    def chat(self):
        while "[exit]" not in (user_message := self._input()):
            self.add_log("ユーザー", user_message)

            prompt = (
                self.make_prompt()
                + "<NL>"
                + "システム: "
            )

            token_ids = self.tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

            output = ""
            for word in self.gradually_generate(token_ids):
                print(word, end='', flush=True)
                output += word
            print()

            self.add_log("システム", output)

    def _input(self):
        s = input("> ").replace("\r", "").replace("\n", "")
        return s

    def make_prompt(self):
        prompt = [
            f"{uttr['speaker']}: {uttr['text']}"
            for uttr in self.log
        ]
        prompt = "<NL>".join(prompt)
        return prompt

    def add_log(self, role, text):
        self.log.append({
            "speaker": role,
            "text": text
        })

    def gradually_generate(self, token_ids):
        token_ids = token_ids.to(self.model.device)
        for _ in range(self.max_length):
            with torch.no_grad():
                outputs = self.model(token_ids)

            logits = outputs.logits
            indices_to_remove = logits < torch.topk(logits, self.k)[0][..., -1, None]
            logits[indices_to_remove] = float('-inf')
            probs = torch.nn.functional.softmax(logits[..., -1, :], dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            token_ids = torch.cat((token_ids, next_token_id), dim=-1)

            output_str = self.tokenizer.decode(next_token_id[0])

            yield output_str.replace("<NL>", "\n")

            if "</s>" in output_str:
                break

In [13]:
# チャットを開始します
bot = RinnaChatBot()
bot.chat()

> 今日も暑いですね
はい、本当に夏です。本当に暑いです。そして、毎日、日ごとに気温が上昇しています。本当に大変な暑さです。体調管理に気をつけて、熱中症にならないように、水分を十分に補給することが大切です。また、運動や外出の際には、日焼け対策をしっかり行う必要があります。暑くて日差しが強いため、サングラスや帽子、日焼け止めなどの紫外線対策も、健康と美容のために重要です。さらに、熱中症対策や熱中症の進行を防止するために、定期的に水分補給をしたり、涼しい服装をしたりすること
> 熱中症には気をつけないといけないですね
はい、熱中症は大変な病気で、重症になると命にもかかわることがあります。また、予防するために、定期的に運動や外出をしたり、水分補給をしたりすることが大切です。さらに、外出時には、日差し対策をしっかりと行い、熱中症を防止することが大切です。</s>
> 水分補給のほかに気をつけたほうがいいことはありますか
はい、カリウムを含む食品を摂取することが重要です。カリウムは、野菜、ナッツ、全粒穀物、豆類、ヨーグルトに豊富に含まれています。カリウムが多く、食感がよい食品には、レタス、ほうれん草、芽キャベツ、芽キャベツ、にんにく、トマトがあります。バナナや柑橘類にもカリウムが豊富に含まれています。カリウムは、筋肉を収縮させる役割も果たし、運動や減量に役立つことがあります。</s>
> [exit]
